# Task 1



# Imports

In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
import librosa
from tqdm import tqdm
sns.set()
from sklearn.model_selection import train_test_split
from sklearn import metrics, preprocessing
from sklearn.metrics import confusion_matrix

In [ ]:
!pip install essentia

     |████████████████████████████████| 12.0MB 285kB/s 


In [ ]:
import essentia as es
from essentia.standard import *

# Colab-Drive Linking

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
address_folder = "/content/drive/MyDrive/genres" 

# Global Constants 

In [ ]:
TOTAL_SAMPLES = 661500 #30sec * 22050Hz


# Load Audio Files

In [ ]:
set1 = ['classical', 'hiphop', 'metal', 'disco', 'reggae']
set2 = ['pop', 'rock', 'country', 'jazz', 'blues']

def load_files(address, set):
  labels = []
  audio = []
  # for i in tqdm(len(set)):
  #   genre = set[i]
  for genre in tqdm(set):
    print("Genre: {}".format(genre))
    path_of_audios = librosa.util.find_files(address + "/" + genre)
    for each in tqdm(path_of_audios):
      x , sr = librosa.load(each)
      audio.append(x)
      labels.append(genre)
  
  return audio,labels



In [ ]:
def get_rhythmic_files(address, set):
  audio = []
  for genre in tqdm(set):
    print("Genre: {}".format(genre))
    path_of_audios = librosa.util.find_files(address + "/" + genre)
    for each in tqdm(path_of_audios):
      audio.append(MonoLoader(filename=each)())
  return np.array(audio, dtype=object)

In [ ]:
files_set1,labels_set1 = load_files(address_folder,set1)
files_set2,labels_set2 = load_files(address_folder,set2)


In [ ]:
files_set1_numpy = np.array(files_set1, dtype = object)

files_set2_numpy = np.array(files_set2, dtype = object)


In [ ]:
labels_set1_numpy = np.array(labels_set1, dtype = object)
labels_set2_numpy = np.array(labels_set2, dtype = object)

In [ ]:
np.save('/content/drive/MyDrive/MMT/files_set1',files_set1_numpy)
np.save('/content/drive/MyDrive/MMT/files_set2', files_set2_numpy)


In [ ]:
np.save('/content/drive/MyDrive/MMT/labels_set1',labels_set1_numpy)
np.save('/content/drive/MyDrive/MMT/labels_set2',labels_set2_numpy)

In [ ]:
files_set1_numpy = np.load('/content/drive/MyDrive/MMT/files_set1.npy',allow_pickle=True)
files_set2_numpy = np.load('/content/drive/MyDrive/MMT/files_set2.npy',allow_pickle=True)


In [ ]:
labels_set1_numpy = np.load('/content/drive/MyDrive/MMT/labels_set1.npy',allow_pickle=True)
labels_set2_numpy = np.load('/content/drive/MyDrive/MMT/labels_set2.npy',allow_pickle=True)

In [ ]:
rhythm_files_set1_numpy = get_rhythmic_files(address_folder,set1)
rhythm_files_set2_numpy = get_rhythmic_files(address_folder,set2)

  0%|          | 0/100 [00:00<?, ?it/s]

Genre: classical



 20%|██        | 1/5 [01:44<06:59, 104.98s/it]

Genre: hiphop



 40%|████      | 2/5 [03:22<05:08, 102.72s/it]

...


 60%|██████    | 3/5 [04:56<03:16, 98.18s/it]

Genre: jazz



 80%|████████  | 4/5 [06:32<01:37, 97.70s/it]

Genre: blues



100%|██████████| 5/5 [08:11<00:00, 98.38s/it]


In [ ]:
#load rhythmic files

with open('/content/drive/MyDrive/rhythm/set1/audio_rhythm_set1.pkl', 'rb') as f:
  rhythm_files_set1_numpy = np.array(pickle.load(f), dtype=object)
with open('/content/drive/MyDrive/rhythm/set2/audio_rhythm_set1.pkl', 'rb') as f:
  rhythm_files_set2_numpy = np.array(pickle.load(f), dtype=object)


# Feature Extraction

In [ ]:
def get_timbral_features(files,tw,aw):

  total_parts_file = int(30/tw)
  samples_per_tw = int(TOTAL_SAMPLES/total_parts_file)
  aw_per_tw = int(tw/aw)
  samples_per_aw_per_tw = int(samples_per_tw/aw_per_tw)
  ans_feature_mat = []
  for file in tqdm(files):
    spectral_centroid = np.array(librosa.feature.spectral_centroid(y=file, sr=22050, n_fft=samples_per_aw_per_tw, hop_length=samples_per_aw_per_tw)[0])
    spectral_rolloff = np.array(librosa.feature.spectral_rolloff(y=file, sr=22050, n_fft=samples_per_aw_per_tw, hop_length=samples_per_aw_per_tw)[0])
    spectral_flux = np.array(librosa.onset.onset_strength(y=file, sr=22050, center=True))
    zero_crossing = librosa.feature.zero_crossing_rate(y=file, frame_length=samples_per_aw_per_tw, hop_length=samples_per_aw_per_tw)
    mfcc = np.array(librosa.feature.mfcc(y=file, sr=22050, n_fft=samples_per_aw_per_tw, hop_length=samples_per_aw_per_tw)[:5,:])
    rmse = np.array(librosa.feature.rms(file, frame_length = samples_per_aw_per_tw, hop_length = samples_per_aw_per_tw)[0])

    # print(type(zero_crossing))
    total_aw = len(spectral_centroid)
    temp_mat = np.zeros((total_aw,17))
    feature_row = [] 
    step = aw_per_tw
    for texture_index in range(total_parts_file):
      temp_mat[texture_index][0] = np.mean(spectral_centroid[step*texture_index:step*(texture_index+1)])
      temp_mat[texture_index][1] = np.var(spectral_centroid[step*texture_index:step*(texture_index+1)])
            
      temp_mat[texture_index][2] = np.mean(spectral_rolloff[step*texture_index:step*(texture_index+1)])
      temp_mat[texture_index][3] = np.var(spectral_rolloff[step*texture_index:step*(texture_index+1)])

      temp_mat[texture_index][4] = np.mean(zero_crossing[0][step*texture_index:step*(texture_index+1)])
      temp_mat[texture_index][5] = np.var(zero_crossing[0][step*texture_index:step*(texture_index+1)])

      texture_window = rmse[step*texture_index:step*(texture_index+1)]
      rmse_mean = np.mean(rmse[step*texture_index:step*(texture_index+1)])
      low_energy = len(texture_window[np.where(texture_window < rmse_mean)])/float(len(texture_window))
      temp_mat[texture_index][6] = low_energy

      texture_window_mean = np.mean(mfcc[:,step*texture_index:step*(texture_index+1)], axis=1)
      texture_window_var = np.var(mfcc[:,step*texture_index:step*(texture_index+1)], axis=1)

      for row,col in enumerate(range(7,17,2)):
        temp_mat[texture_index][col] = texture_window_mean[row]
        temp_mat[texture_index][1+col] = texture_window_var[row]

    feature_row = np.mean(temp_mat, axis=0).tolist() 
    feature_row += [np.mean(spectral_flux), np.var(spectral_flux)]

    feature_row = np.array(feature_row)
    ans_feature_mat.append(feature_row)
  
  return ans_feature_mat
    

In [ ]:
def get_rhythm_features(files):
  # Compute beat positions and BPM
  rh_features_mat = []
  for audio in tqdm(files):
    temp_row = []
    rhythm_extractor = RhythmExtractor2013(method="multifeature")
    bpm, beats, beatsconfidence, _, beats_intervals = rhythm_extractor(audio)
    peak1_bpm, peak1_weight, peak1_spread, peak2_bpm, peak2_weight, peak2_spread, histogram = BpmHistogramDescriptors()(beats_intervals)
    temp_row += [peak1_weight/np.sum(histogram), peak2_weight/np.sum(histogram), peak2_weight/peak1_weight, peak1_bpm, peak2_bpm, sum(histogram)]
    rh_features_mat.append(np.array(temp_row))
  
  return np.array(rh_features_mat)

In [ ]:
# get pitch features
import os
from xml.etree import ElementTree
def get_pitch_features(address_filename):
    file_name = address_filename
    full_file = os.path.abspath(os.path.join("", file_name))
    dom = ElementTree.parse(full_file)
    audios = dom.findall("data_set")

    all_features_matrix = []
    for audio in audios:
        all_features =  audio.findall("feature")

        all_feature_one_file = []
        for f in all_features:
            values = f.findall("v")
            for v in values:
                all_feature_one_file.append(float(v.text))
            
        all_features_matrix.append(np.array(all_feature_one_file, dtype=object))

    return np.array(all_features_matrix, dtype=object)